In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

In [2]:
def make_predictions(model,scaler_y,trainX,trainY,testX,testY,n_steps_out,len_output_features):
    
    # make predictions
    trainPredict = model.predict(trainX)
    trainPredict = trainPredict.reshape(trainPredict.shape[0]*n_steps_out,len_output_features)
    testPredict  = model.predict(testX)
    testPredict  = testPredict.reshape(testPredict.shape[0]*n_steps_out,len_output_features)
    
    # invert predictions
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY_ = scaler_y.inverse_transform(trainY.reshape(trainY.shape[0]*n_steps_out,len_output_features))
    
    testPredict = scaler_y.inverse_transform(testPredict)
    testY_ = scaler_y.inverse_transform(testY.reshape(testY.shape[0]*n_steps_out,len_output_features))
        
    return trainPredict,trainY_,testPredict,testY_

def get_metrics(trainY,trainPredict,testY,testPredict):
    
    trainMAPE  = Ep.MAPE(trainPredict,trainY)
    testMAPE  = Ep.MAPE(testPredict,testY)
    
    return trainMAPE,testMAPE

# DATASET

## Load Dataset

In [3]:
data_diaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Diaria.xlsx')
data_diaria = pd.read_excel(data_diaria_path)
data_diaria = data_diaria.set_index('Fecha')

In [4]:
nombre_series = data_diaria.columns.values

In [5]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [6]:
data_diaria.shape, precio_bolsa.shape

((7365, 119), (176760, 1))

## Build Window

In [7]:
d = 'All'
time_split = '2020-01-01'
n_steps_in = 30
n_steps_out=24
overlap = 1
inputs_columns = nombre_series
output_columns = ['$kWh']

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

trainX, trainY, testX, testY, scaler_x,scaler_y, dataset_x, dataset_y = Ep.SplitTimeseriesMultipleTimesBackAhead_differentTimes(
    df_x=data_diaria,
    df_y=precio_bolsa,
    day = d, 
    TimeSplit = time_split,
    n_steps_out=n_steps_out,
    n_steps_in =n_steps_in, 
    overlap = overlap,
    input_features=inputs_columns,
    output_features=output_columns)

trainX.shape,trainY.shape,testX.shape,testY.shape

((7244, 30, 119), (7244, 24, 1), (91, 30, 119), (91, 24, 1))

In [8]:
"""
log_dir = os.path.join('logs','Optimización')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=2,
                                                          write_graph=False,
                                                          update_freq='epoch
"""
    
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-4,
                                                          patience=0,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  mode='min')

#callbacks = [callback_tensorboard,callback_reduce_lr,early_stopping]
callbacks = [callback_reduce_lr,early_stopping]

In [9]:
INPUT_SHAPE = (n_steps_in,len_input_features)

arquitectura31 = CustomHyperModel.Arquitectura31(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura32 = CustomHyperModel.Arquitectura32(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura33 = CustomHyperModel.Arquitectura33(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura34 = CustomHyperModel.Arquitectura34(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura35 = CustomHyperModel.Arquitectura35(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura36 = CustomHyperModel.Arquitectura36(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura37 = CustomHyperModel.Arquitectura37(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura38 = CustomHyperModel.Arquitectura38(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

arquitectura39 = CustomHyperModel.Arquitectura39(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura40 = CustomHyperModel.Arquitectura40(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura41 = CustomHyperModel.Arquitectura41(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura42 = CustomHyperModel.Arquitectura42(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura43 = CustomHyperModel.Arquitectura43(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura44 = CustomHyperModel.Arquitectura44(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura45 = CustomHyperModel.Arquitectura45(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura46 = CustomHyperModel.Arquitectura46(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

In [10]:
arq_list = [arquitectura31,arquitectura32,arquitectura33,arquitectura34,
            arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""
arq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""

'\narq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,\n            arquitectura39,arquitectura40,arquitectura41,arquitectura42,\n            arquitectura43,arquitectura44,arquitectura45,arquitectura46]\n'

In [11]:
arq_idx = 31
arq_best_models = dict()

for arq in arq_list:
    
    bayesian_tuner = BayesianOptimization(
        arq,
        objective='val_mean_absolute_percentage_error',
        num_initial_points=1,
        max_trials=10,
        directory='my_dir',
        project_name=str(arq_idx)
    )
    
    # Overview of the task
    bayesian_tuner.search_space_summary()
    
    # Performs the hyperparameter tuning
    search_start = time.time()
    bayesian_tuner.search(x=trainX,y=trainY,
                      epochs=200,
                      validation_data=(testX,testY),
                      callbacks=callbacks)
    search_end = time.time()
    elapsed_time = search_end - search_start
    
    print('Tiempo Total Transcurrido {}'.format(elapsed_time))
        
    dict_key = 'Arquitectura {}'.format(arq_idx)

    arq_best_models[dict_key] = dict()
    bs_model = bayesian_tuner.oracle.get_best_trials(1)[0]
    
    model = bayesian_tuner.get_best_models(num_models=1)[0]
    
    trainPredict,trainY_true,testPredict,testY_true = make_predictions(model,scaler_y,trainX,trainY,testX,testY,
                                                             n_steps_out,len_output_features)
    
    trainMAPE,testMAPE = get_metrics(trainY_true,trainPredict,testY_true,testPredict)

    arq_best_models[dict_key]['Score'] = bs_model.score
    arq_best_models[dict_key]['Tiempo Scaneo'] = elapsed_time
    arq_best_models[dict_key]['Mape Train'] = trainMAPE
    arq_best_models[dict_key]['Mape Test'] = testMAPE

    if bs_model.hyperparameters.values:
        for hp, value in bs_model.hyperparameters.values.items():
            arq_best_models[dict_key][hp] = value
    
    arq_idx += 1

Trial 10 Complete [00h 00m 32s]
val_mean_absolute_percentage_error: 25.055665969848633

Best val_mean_absolute_percentage_error So Far: 19.48134994506836
Total elapsed time: 00h 07m 03s
INFO:tensorflow:Oracle triggered exit
Tiempo Total Transcurrido 423.5437705516815


In [12]:
import json
with open('BestModels25.json', 'w') as outfile:
    json.dump(arq_best_models, outfile)

In [13]:
arq_best_models

{'Arquitectura 31': {'Score': 25.512014389038086,
  'Tiempo Scaneo': 0.0039865970611572266,
  'Mape Train': 0.39236496405007487,
  'Mape Test': 0.2301252116783175,
  'lstm_units_layer_1': 64,
  'kernel_regularizer_layer_1': 0.0975,
  'dropout_regularizer_layer_1': 0.0,
  'lstm_units_layer_2': 512,
  'kernel_regularizer_layer_2': 0.03,
  'dropout_regularizer_layer_2': 0.0,
  'dense_layer': 120,
  'dense_layer_activation': 'relu',
  'dropout_dense': 0.0,
  'learning_rate': 0.0001},
 'Arquitectura 32': {'Score': 55.151371002197266,
  'Tiempo Scaneo': 400.6307144165039,
  'Mape Train': 1.1732883010689814,
  'Mape Test': 0.5161153753285608,
  'gru_units_layer_1': 384,
  'kernel_regularizer_layer_1': 0.0525,
  'dropout_regularizer_layer_1': 0.36,
  'gru_units_layer_2': 256,
  'kernel_regularizer_layer_2': 0.08249999999999999,
  'dropout_regularizer_layer_2': 0.0,
  'dense_layer': 96,
  'dense_layer_activation': 'tanh',
  'dropout_dense': 0.18,
  'learning_rate': 0.0003715717019676744},
 'Arq

In [ ]:
bayesian_tuner = BayesianOptimization(
    arquitectura38,
    objective='val_mean_absolute_percentage_error',
    num_initial_points=1,
    max_trials=15,
    directory='my_dir',
    project_name='tuning_38'
)

# Overview of the task
bayesian_tuner.search_space_summary()

# Performs the hyperparameter tuning
search_start = time.time()
bayesian_tuner.search(x=trainX,y=trainY,
                      epochs=200,
                      validation_data=(testX,testY),
                      callbacks=callbacks)
search_end = time.time()
elapsed_time = search_end - search_start

In [ ]:
print('Tiempo Total Transcurrido {}'.format(elapsed_time))

# Show a summary of the search
bayesian_tuner.results_summary()

In [ ]:
# Retrieve the best model.
best_model = bayesian_tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, mae, mape, smape = best_model.evaluate(x=testX,y=testY)

In [ ]:
best_model.to_json()